# Topic Classification

In this notebook we use the output from the topic model and our immersion journal/manual to subset a training set of tweets about the Australian Bushfires. 

In [1]:
#Importing relevant packages
import numpy as np 
import pandas as pd
from tqdm import tqdm #to create a progress bar
#Machine learning packages
from sklearn.linear_model import LogisticRegression
#Packages to create DFM
from sklearn.feature_extraction.text import CountVectorizer
#Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns
#Word embeddings
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
#Gary King et. al key-words
from keyword_algorithm import *
#Remove unwarranted warnings
pd.options.mode.chained_assignment = None  # default='warn'

## Implementation of Gary Kings semi-automated keyword retrival

In [2]:
#Load and prepare data
data = pd.read_csv("data/final_df.csv", index_col=0)
data = data.dropna(subset = ["lemmas"]).reset_index(drop = True)
data["index_col"] = data.index
#data.to_csv("data/query_df.csv")

In [4]:
#Download model from https://fasttext.cc/docs/en/english-vectors.html
model_dir = "data/wiki-news-300d-1M.vec"
fasttext = KeyedVectors.load_word2vec_format(model_dir)

In [9]:
class QueryBuilder:
    
    def __init__(self, emb_model):
        
        self.query = Keywords()
        #Load the data. Change path if necessary
        path = 'data/query_df.csv'
        self.query.LoadDataset(path, text_colname='lemmas', 
                    date_colname="created_at", id_colname="index_col")
        #Load Gensim word embeddings model 
        self.we = emb_model
            
    def get_keywords(self, its = 2, top_n = 10, refkeys = [], tarkeys = [], algorithms = ['nbayes', 'logit'], 
                     date_start = "2019-06-01", date_end = "2020-05-30"):
        """
        Loops over King. et. al algorithm to extract relevant keywords used for building a 
        boolean query to subset relevant Tweets in the dataset.
        ---------
        arguments:
            - its: Iterations to run the algorithm
            - top_n: integer of how many of the most predictive keywords to extract in each iteration
            - refkeys: list of initial keywords used to create reference set of tweets
            - tarkeys: list of initial keywords used to limit the search set
            - algorithms: list of classifiers to run for extracting keywords 
            - date_start: y/m/d of start date for relevant tweets
            - date_end: y/m/d of end date for relevant tweets
        -----------    
        returns:
            - dictionary of accepted, rejected and nontarget keywords
        """
        
        accepted_keywords = []
        rejected_keywords = []
        nontarget_keywords = []
        
        #Begin loop for mining search set
        for it in range(its):
            print("-"*66)
            print(f"STARTING ITERATION: {it}!")
            if it == 0:
                print("INITIAL REFERENCE KEYS: {refkeys} \n INITIAL TARGET KEYS: {tarkeys}")
            print("-"*66)
            
            #Build reference set of tweets
            self.query.ReferenceSet(any_words=refkeys, date_start=date_start, date_end=date_end)
            
            #Use accepted keys as search keys if not the first iteration
            if it > 0:
                self.query.SearchSet(any_words = accepted_keywords, 
                                     date_start=date_start, date_end=date_end)
            else:
                self.query.SearchSet(any_words = tarkeys, 
                                     date_start=date_start, date_end=date_end)
            
            
            #Run King algorithm to find keywords.
            self.query.ProcessData(stem = False, keep_twitter_symbols=False,
                                   remove_wordlist=refkeys)
            self.query.ReferenceKeywords()
            self.query.ClassifyDocs(min_df=5, ref_trainprop=1, algorithms=algorithms)
            self.query.FindTargetSet()
            self.query.FindKeywords()
            
            #Extract target keywords from algorithm results
            target_keywords = self.query.target_keywords[:top_n]
            #Also get the reference set keywords to loop over
            target_keywords += self.query.reference_keywords[:top_n]
            #Append unique nontarget keywords to list of nontarget keys
            for nonkey in self.query.nontarget_keywords[:100]:
                if nonkey not in nontarget_keywords:
                    nontarget_keywords.append(nonkey)
            
            #Loop over each relevant keyword from reference and found target keywords
            for keyword in target_keywords:
                #Check if keyword has already been rejected or accepted
                if keyword in accepted_keywords or keyword in rejected_keywords:
                    continue
                else:
                    inp = input(f"Keep {keyword.upper()} yes or no?")
                    if inp == "yes":
                        accepted_keywords.append(keyword)
                        #get similar keywords through most similar pretrained embeddings
                        inp2 = input(f"Look at {keyword.upper()}'s most similar word embeddings, yes or no?")
                        if inp2 == "yes":
                            #Look if keyword exist in embedding model dictionary
                            try:
                                embeddings = [emb[0] for emb in self.we.most_similar(keyword)]
                                for emb in embeddings:
                                    if emb.lower() in accepted_keywords or emb.lower() in rejected_keywords:
                                        continue
                                    else:
                                        inp3 = input(f"Keep embedding {emb.upper()} yes or no?")
                                        if inp3 == "yes":
                                            accepted_keywords.append(emb)
                                        elif inp3 =="no":
                                            rejected_keywords.append(emb)
                            except:
                                print(f"{keyword.upper()} embedding not present in Model!")
                                pass
                        elif inp2 == "no":
                            pass
                    elif inp == "no":
                        rejected_keywords.append(keyword)
                        
            #Add custom keyword(s) in the end of the loop. Either as list or single keyword
            inp4 = input(f"Do you wish to add any further keywords? If yes, Type keyword: ")
            if inp4:
                if isinstance(inp4, list):
                    [accepted_keywords.append(key) for key in inp4]
                else:
                     accepted_keywords.append(inp4)
            else:
                pass
            
            print("-"*66)
            print(" "*20, f"CURRENT KEYWORDS AFTER ITTERATION {it}")
            print("-"*66)
            print(f"ACCEPTED: \n {accepted_keywords}")
            print(f"REJECTED: \n {rejected_keywords}")
            
        
        keywords = {"accepted_keys":accepted_keywords, "rejected_keys":rejected_keywords,"nontarget_keys":nontarget_keywords}
        
        return keywords
        
        
        

In [10]:
query = QueryBuilder(fasttext)

Keyword object initialized.
Loaded corpus of size 148540 in 2.21 seconds.


In [ ]:
keywords

In [130]:
keywords = query.get_keywords(2, top_n=10, refkeys=["#bushfire", "#bushfires", "bushfire", "bushfires"], 
                                 tarkeys = ["fire"])

------------------------------------------------------------------
STARTING ITERATION: 0!
INITIAL REFERENCE KEYS: {refkeys} 
 INITIAL TARGET KEYS: {tarkeys}
------------------------------------------------------------------
Loaded reference set of size 1682 in 2.09 seconds.
Loaded search set of size 1368 in 0.52 seconds.
Time to process corpus: 0.49 seconds

4159 reference set keywords found.

Document Term Matrix: 3050 by 1495 with 37198 nonzero elements

Time to get document-term matrix: 0.05 seconds

Ref training size: 1682; Search training size: 451; Training size: 2133; Test size: 1368

Time for Naive Bayes: 0.01 seconds
Time for Logit: 0.04 seconds
473 documents in target set
895 documents in non-target set
253 target set keywords found
185 non-target set keywords found
Keep SUPPORT yes or no? yes
Look at SUPPORT's most similar word embeddings, yes or no? yes
Keep embedding SUPPORTING yes or no? yes
Keep embedding SUPPPORT yes or no? yes
Keep embedding SUPPORTED yes or no? yes
Ke

## Mlogit Model - Average Marginal Effect Estimation 

Here we use the key-words from Kings model to subset tweets. Next we use this subset to predict the party of the tweet and analyze the coefficients

In [84]:
class MlogitMargins:
    """
    Calculates marginal effect of multinomial logit coefficients with bootstraped confidence interval.
    See https://github.com/alicehwu/gendered_language/blob/master/gendered_language_2018.pdf for a 
    reference on a similar approach but with binary classification.
    """
    def __init__(self, X, y):
        #Define data
        self.vect = CountVectorizer(ngram_range=(1,2), min_df = 5)
        self.X = self.vect.fit_transform(X)
        self.y = y
        print("Fitting model and calculating margins...")
        #Fit model and calculate average marginal effects
        self.margins, self.fitted_model = self.avg_margins(self.X, self.y)
    
    def avg_margins(self, X, y):
        """
        Calculates average marginal effect of coefficients in multinomial logit model, where each coefficient is 
        a word token and is calculated as:
        avg_margin_jk =  1/N * sum_i{sum{P(y_i = j) * [beta_jk - sum_m(P(y_i = m) * beta_jm)]} for each token k and class j.
        -----------
        Returns:
            - DataFrame with average margins of shape classes_j * tokens_k
            - The fitted mlogit model 
        """
        #Fit the model
        model = self.fit_model(X, y)
        #Get probabilities for each obs i belonging to class j. shape = j * N
        probas = model.predict_proba(X)
        #Get coefficients. Shape j_classes * k_coefficients
        betas = model.coef_
        
        avg_margins = {}
        for j in tqdm(range(len(model.classes_))):
            for k in range(betas.shape[1]):
                dydw_jk = 0 
                for i in range(probas.shape[0]):
                    dydw_jk +=  probas[i,j] * (betas[j,k] - sum([probas[i,m] * betas[m,k] for m in range(len(model.classes_))]))

                avg_margin = 1 / probas.shape[0] * dydw_jk
                if model.classes_[j] not in avg_margins.keys():
                            avg_margins[model.classes_[j]] = [avg_margin]
                else:
                    avg_margins[model.classes_[j]].append(avg_margin)
        
        #Extract the token name corresponding to the avg margins   
        avg_margins["token"] = self.vect.get_feature_names()
        margins_df = pd.DataFrame(avg_margins)
        
        return margins_df, model

    def avg_margins2(self, X, y):
        """
        Calculates average marginal effect of coefficients in multinomial logit model, where each coefficient is 
        a word token and is calculated as:
        avg_margin_jk = beta_jk * 1/N * sum{P(y_i = J) * 1-P(y_i = J)} for each token k and class j.
        See: https://math.stackexchange.com/questions/863258/deriving-marginal-effects-in-multinomial-logit-model
        -----------
        Returns:
            - DataFrame with average margins of shape classes_j * tokens_k
            - The fitted mlogit model 
        """
        #Fit the model
        model = self.fit_model(X, y)
        #Get probabilities for each obs i belonging to class j. shape = j * N
        probas = model.predict_proba(X)
        #Get coefficients. Shape j_classes * k_coefficients
        betas = model.coef_

        margins = {}
        #Loop over each class 
        for class_j in range(len(model.classes_)):
            #Extract corresponging betas. shape 1*k
            for beta_jk in betas[class_j]:
                #Calculate avg margins for the jth class and kth beta (token)
                margins_jk = beta_jk * 1/probas.shape[0] * (probas[:,class_j] * (1 - probas[:,class_j])).sum()
                #Append to dictionary
                if model.classes_[class_j] not in margins.keys():
                    margins[model.classes_[class_j]] = [margins_jk]
                else:
                    margins[model.classes_[class_j]].append(margins_jk)
        
        #Extract the token name corresponding to the avg margins   
        margins["token"] = self.vect.get_feature_names()
        margins_df = pd.DataFrame(margins)
        
        return margins_df, model
    
    def fit_model(self, X, y, max_iter = 10000, penalty = "l1", class_weight = "balanced",
                            verbose = False, fit_intercept = True, multi_class = "multinomial",
                            C = 1):
        
        """
        Fits sklearn multinomial logistic model on data.
        """

        mlogit = LogisticRegression(random_state=42, penalty=penalty, solver="saga", 
                                                           max_iter = max_iter, class_weight = class_weight,
                                                           fit_intercept=fit_intercept, multi_class=multi_class,
                                                           verbose=verbose, C = C).fit(X, y)  

        return mlogit
    
    def bootstrap_ci(self, alpha = 0.05, n_samples = 500, sample_prop = 0.4):
        """
        Uses bootstraping to calculate confidence interval around average marginal effects estimate.
        -------------
        Arguments:
            - alpha: deterimines confidence level of the interval
            - n_samples: amount of bootstrap samples
            - sample_prop: bootstramp sample size as proportion of original sample
        -------------
        Return:
            - Pandas dataframe with confidence interval for average marginal effect
              of each coefficient.
        """
        statistics = [] #List for bootstrap results
        
        #Define the bootstrap sample size based on proportion of total
        n_size = int(self.X.shape[0] * sample_prop)
        print(f"Number of obs in bootstrap samples {n_size}...")
        start = time.time()
        for i in tqdm(range(n_samples)):
            #Draw random sample from X and y with replacement
            idx = np.random.choice(np.arange(self.X.shape[0]), n_size, replace=True)
            X_sample = self.X[idx]
            y_sample = self.y[idx]
            #Calculate marginal effect for sample
            margins = self.avg_margins(X_sample, y_sample)[0]
            statistics.append(margins)
        
        #Join the resulting dataframes of margins
        statistics = pd.concat(statistics)
        print(f"Boostraping Done. Calculating {1-alpha}% confidence interval...")
        #From bootstrap results get lower and upper CI limits based on alpha
        statistics = statistics.groupby("token").quantile([alpha, 1-alpha]).reset_index()        
        print("Time to complete: ", time.time() - start)

        return statistics
    

In [85]:
#NOTE: This data is just for testing - not final subset.
mlogit_data = data.loc[(data["lemmas"].str.contains("bushfire|bushfires|#bushfire|#bushfires|fire|flames|firemen") == True) & 
                                         (data["lemmas"].str.contains("corona|covid") == False) & 
                                         (data["created_at"] >= "2019-06-01") & 
                                         (data["created_at"] <= "2020-06-01")]

#Reduce classes to predict. Otherwise bad model performance.
mlogit_data = mlogit_data.loc[mlogit_data["party"].isin(["Australian Greens",
                                                                                                   "Australian Labor Party", 
                                                                                                   "Liberal Party of Australia",
                                                                                                    ])].dropna(subset = ["lemmas"]).reset_index(drop = True)

In [86]:
X, y = mlogit_data["lemmas"], mlogit_data["party"]
mlogit = MlogitMargins(X, y)

Fitting model and calculating margins...


100%|██████████| 3/3 [00:33<00:00, 11.16s/it]


In [88]:
mlogit.margins.sort_values("Liberal Party of Australia", ascending=False).head(20)

,Australian Greens,Australian Labor Party,Liberal Party of Australia,token
385,-0.038275,-0.211655,0.249929,courage
836,-0.037718,-0.208577,0.246294,initial
1054,-0.036854,-0.203797,0.240650,network
1053,-0.084561,-0.136502,0.221064,need take
17,-0.032217,-0.178154,0.210371,additional
1466,-0.031822,-0.175974,0.207796,telecommunication
528,-0.030986,-0.171349,0.202335,ensure
859,-0.030561,-0.168997,0.199558,join
431,-0.029335,-0.162218,0.191552,defence force
109,-0.028565,-0.157960,0.186525,attend


In [83]:
mlogit.margins.sort_values("Australian Greens", ascending=False).head(20)

,Australian Greens,Australian Labor Party,Liberal Party of Australia,token
332,0.346922,0.000000,0.000000,climateemergency
336,0.178290,0.000000,-0.178607,coal
325,0.167382,0.000000,-0.045300,climate
203,0.158005,0.000000,0.000000,building
1533,0.155544,0.000000,0.000000,town
1161,0.155027,-0.053213,0.000000,port
150,0.154716,0.000000,0.000000,beach
405,0.136338,0.000000,0.000000,cut
594,0.133839,-0.037640,0.000000,fight bushfires
912,0.121663,0.000000,0.000000,literally


## Testing

In [17]:
vect = CountVectorizer(ngram_range=(1,2), min_df = 5)
X = vect.fit_transform(X)
feature_names = vect.get_feature_names()

In [58]:
def fit_model(X, y, max_iter = 10000, penalty = "l1", class_weight = "balanced",
                            verbose = False, fit_intercept = True, multi_class = "multinomial",
                            C = 1):
        
        """
        Fits sklearn multinomial logistic model on data.
        """
    
        
        mlogit = LogisticRegression(random_state=42, penalty=penalty, solver="saga", 
                                                           max_iter = max_iter, class_weight = class_weight,
                                                           fit_intercept=fit_intercept, multi_class=multi_class,
                                                           verbose=verbose, C = C).fit(X, y)  
        return mlogit

In [59]:
mlogit  = fit_model(X, y)

In [60]:
probas = mlogit.predict_proba(X)
betas = mlogit.coef_

In [61]:
print(probas.shape, betas.shape)

(2044, 3) (3, 1658)


In [62]:
probas.shape

(2044, 3)

In [63]:
avg_margins = {}
for j in tqdm(range(len(mlogit.classes_))):
    for k in range(betas.shape[1]):
        dydx = 0 
        for i in range(probas.shape[0]):
            dydx +=  probas[i,j] * (betas[j,k] - sum([probas[i,m] * betas[m,k] 
                                                      for m in range(len(mlogit.classes_))]))
        
        avg_margin = 1 / probas.shape[0] * dydx
        if mlogit.classes_[j] not in avg_margins.keys():
                    avg_margins[mlogit.classes_[j]] = [avg_margin]
        else:
            avg_margins[mlogit.classes_[j]].append(avg_margin)
        

100%|██████████| 3/3 [00:31<00:00, 10.56s/it]


In [64]:
test_df = pd.DataFrame(avg_margins)
test_df["term"] = feature_names

In [66]:
test_df.sort_values("Australian Greens", ascending=False).head(15)

,Australian Greens,Australian Labor Party,Liberal Party of Australia,term
332,0.346922,-0.273272,-0.073650,climateemergency
1307,0.206547,-0.330150,0.123603,satellite
336,0.205642,0.010815,-0.216457,coal
1161,0.176395,-0.175329,-0.001066,port
325,0.174319,-0.093485,-0.080834,climate
561,0.164376,-0.228100,0.063724,experience
203,0.158005,-0.124461,-0.033544,building
738,0.155771,-0.270009,0.114238,green
1533,0.155544,-0.122523,-0.033021,town
150,0.154716,-0.121871,-0.032845,beach


In [42]:
count = 0
for i in betas[2]:
    count += 1
count

1658

In [ ]:
margins = {}
        #Loop over each class 
        for class_j in range(len(model.classes_)):
            #Extract corresponging betas. shape 1*k
            for beta_jk in betas[class_j]:
                #Calculate avg margins for the jth class and kth beta (token)
                margins_jk = beta_jk * 1/probas.shape[0] * (probas[:,class_j] * (1 - probas[:,class_j])).sum()
                #Append to dictionary
                if model.classes_[class_j] not in margins.keys():
                    margins[model.classes_[class_j]] = [margins_jk]
                else:
                    margins[model.classes_[class_j]].append(margins_jk)
        
        #Extract the token name corresponding to the avg margins   
        margins["token"] = self.vect.get_feature_names()
        margins_df = pd.DataFrame(margins)